<a href="https://colab.research.google.com/github/JeonyuB/Uchan/blob/Hell/%EC%A7%80%EB%8A%A5%ED%98%95%EC%86%8C%ED%94%84%ED%8A%B8%EC%9B%A8%EC%96%B4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
import numpy as np
import pandas as pd
import matplotlib as plt
import music21
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,LSTM,Dropout
import tensorflow as tf


states = [0, 1, 2, 3, 4, 5, 6, 7]

P = np.array([
  [0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
  [0.0, 0.0, 0.5, 0.0, 0.0, 0.5, 0.0, 0.0],
  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.2],
  [0.0, 0.0, 0.0, 0.0, 0.6, 0.0, 0.4, 0.0],
  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0],
  [0.0, 0.1, 0.0, 0.0, 0.0, 0.9, 0.0, 0.0],
  [0.0, 0.2, 0.4, 0.4, 0.0, 0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
])

R = np.array([0, -2, -2, -2, 10, -1, 1, 0])

V = np.zeros(len(states))
threshold = 0.0001

def value_iteration (gamma):

  while True:
    V_new = np.copy(V)

    for s in states:
      V_new[s] = R[s] + gamma * np.sum([P[s, s_prime] * V[s_prime] for s_prime in states])

    if np.max(np.abs(V_new - V)) < threshold:
      V = V_new
      break

    V = V_new
  print("Value function:", V)



4x4 small gridworld

In [26]:
shape = (4, 4)
terminals = [(0, 0), (3, 3)]

numa = 4
actions = ['up', 'down', 'left', 'right']

reward = -1 * np.ones(shape)
for terminal in terminals:
    reward[terminal] = 0

gamma = 1.0

def P(state, action):
    if action == 'up':
        next_state = (max(0, state[0]-1), state[1])
    elif action == 'down':
        next_state = (min(shape[0]-1, state[0]+1), state[1])
    elif action == 'left':
        next_state = (state[0], max(0, state[1]-1))
    elif action == 'right':
        next_state = (state[0], min(shape[1]-1, state[1]+1))
    return next_state

V = np.zeros(shape)

while True:
    delta = 0
    for i in range(shape[0]):
        for j in range(shape[1]):
            if (i, j) in terminals:
                continue
            v = V[i, j]
            V[i,j]= sum((reward[i, j] + gamma * V[P((i, j), a)]) for a in actions)/numa
            delta = max(delta, abs(v - V[i, j]))
    if delta < 1e-4:
        break

optimal_policy = {}
for i in range(shape[0]):
    for j in range(shape[1]):
        if (i, j) in terminals:
            optimal_policy[i, j] = 'terminal'
        else:
            optimal_policy[i, j] = actions[np.argmax([(reward[i,j]
                                                       +gamma*V[P((i,j),a)]) for a in actions])]

print("Optimal policy is:")
for i in range(shape[0]):
    for j in range(shape[1]):
        print(f"({i},{j}): {optimal_policy[i, j]}")

print("\nOptimal value function is:")
for i in range(shape[0]):
    for j in range(shape[1]):
        print(f"V({i},{j}): {V[i, j]}")


Optimal policy is:
(0,0): terminal
(0,1): left
(0,2): left
(0,3): left
(1,0): up
(1,1): up
(1,2): left
(1,3): down
(2,0): up
(2,1): up
(2,2): right
(2,3): down
(3,0): up
(3,1): right
(3,2): right
(3,3): terminal

Optimal value function is:
V(0,0): 0.0
V(0,1): -13.999312424461952
V(0,2): -19.999011518162753
V(0,3): -21.998911992496346
V(1,0): -13.999312424461952
V(1,1): -17.999156254598965
V(1,2): -19.99908388638086
V(1,3): -19.99909436158647
V(2,0): -19.999011518162757
V(2,1): -19.99908388638086
V(2,2): -17.99922696784339
V(2,3): -13.999422844683943
V(3,0): -21.99891199249635
V(3,1): -19.999094361586472
V(3,2): -13.999422844683945
V(3,3): 0.0


Policy iteration 구현

In [34]:
policy={state: np.random.choice(actions) for state in
        [(i,j) for i in range(shape[0]) for j in range(shape[1])] if state not in terminals}

while True:
  while True:
    delta=0
    for i in range(shape[0]):
      for j in range(shape[1]):
        if (i,j) in terminals:
          continue
        v=V[i,j]
        action=policy[(i,j)]
        V[i,j]=reward[i,j]+gamma*V[P((i,j),action)]
        delta=max(delta,abs(v-V[i,j]))
    if delta<2.01:
      break
  policy_stable=True
  for j in range(shape[0]):
    for j in range(shape[1]):
        if (i,j) in terminals:
          continue
        old_action=policy[(i,j)]
        policy[(i,j)]=actions[np.argmax([reward[i,j]
                                         +gamma*V[P((i,j),action)] for action in actions])]
        if old_action!=policy[(i,j)]:
          policy_stable=False
  if policy_stable:
    break